In [48]:
import pandas as pd
from scipy.stats import ttest_ind
from datetime import datetime
import numpy as np
from tqdm.notebook import tqdm
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore')

In [49]:
df_metrics = pd.read_csv('2022-04-01T12_df_web_logs.csv')
df_metrics['date'] = pd.to_datetime(df_metrics['date'])
df_metrics.head(3)

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7


In [50]:
dt_beg = datetime(2022,3,1)
dt_end = datetime(2022,3,8)
df_metrics = df_metrics[(df_metrics['date']>=dt_beg)&(df_metrics['date']<=dt_end)]
df_metrics.head(3)

,user_id,page,date,load_time
885082,434cf2,m,2022-03-01 00:00:02,69.8
885083,80fa93,m,2022-03-01 00:00:06,86.3
885084,434cf2,b,2022-03-01 00:02:20,58.0


In [53]:
users = df_metrics['user_id'].unique()

In [57]:
def process_results(quantile2errors):
    for q, errors in quantile2errors.items():
        error_rate = 1 - np.mean(errors)
        print(f"Quantile {q}: Error rate = {error_rate:.2%}")

In [59]:
sample_size = 1000
effect = 0.01
alpha = 0.05
quantiles = (0.0001, 0.001, 0.01, 0.05, 0.1,)
quantile2errors = {q: [] for q in quantiles}

for _ in tqdm(range(10000)):
    a_users, b_users = np.random.choice(users, (2, sample_size,), False)
    a_values = df_metrics.loc[df_metrics['user_id'].isin(a_users), 'load_time'].values
    b_values = df_metrics.loc[df_metrics['user_id'].isin(b_users), 'load_time'].values
    mean_ = b_values.mean()
    indexes = np.random.choice(np.arange(len(b_values)), int(len(b_values) / 100), False)
    b_values[indexes] += mean_ * effect * len(b_values) / len(indexes)

    for q in quantiles:
        a_values_filtered = a_values[
            (a_values > np.quantile(a_values, q))
            & (a_values < np.quantile(a_values, 1-q))
        ]
        b_values_filtered = b_values[
            (b_values > np.quantile(b_values, q))
            & (b_values < np.quantile(b_values, 1-q))
        ]
        pvalue = stats.ttest_ind(a_values_filtered, b_values_filtered).pvalue
        quantile2errors[q].append(pvalue > alpha)

process_results(quantile2errors)


  0%|          | 0/10000 [00:00<?, ?it/s]

Quantile 0.0001: Error rate = 8.83%
Quantile 0.001: Error rate = 34.58%
Quantile 0.01: Error rate = 45.80%
Quantile 0.05: Error rate = 29.68%
Quantile 0.1: Error rate = 32.14%
